In [1]:
access_token = "XXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXX"
consumer_key = "XXXXXXXXXXXXXX"
consumer_secret = "XXXXXXXXXXXXXX"

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import tweepy
import pymongo,json
from textblob import TextBlob 
import re 
import sys
import pandas as pd
import pprint

class TwitterClient(object): 
    
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
    def get_tweet_sentiment(self, tweet): 
        analysis = TextBlob(self.clean_tweet(tweet))  
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [7]:
class StdOutListener(StreamListener,TwitterClient):
    def __init__(self,maxtweets):
        super(StdOutListener, self).__init__()
        self.maxtweets=maxtweets
        self.ctr=0
    def mongo(self,data):
        connection = pymongo.MongoClient("mongodb://localhost")
        db=connection["Tweets"]
        record = db["tweet_collection"]
        record.insert_one(data)
        connection.close()
        if (self.ctr%1000)==0:
            print('closed')
        return True

    def on_status(self, data):
        tweets=[] 
        parsed_tweet={}
        try: 
                data=json.dumps(data._json)
                data=json.loads(data)
                if data["text"][0:2]=='RT':
                    return 0
                self.ctr=self.ctr+1
                #print (data)
                #print("----------------------------------------------------------------------")
                parsed_tweet['sname']= data["user"]["name"]
                try:
                    parsed_tweet['text'] = data["extended_tweet"]["full_text"]
                    parsed_tweet['sentiment'] = self.get_tweet_sentiment(data["extended_tweet"]["full_text"])
                except:
                    parsed_tweet['text'] = data["text"]
                    parsed_tweet['sentiment'] = self.get_tweet_sentiment(data["text"])
                
                '''if data["retweet_count"] > 0:
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: '''
                tweets.append(parsed_tweet)
                if  self.ctr>self.maxtweets:
                    sys.exit('Limit reached')
                #print(tweets)
                self.mongo(parsed_tweet)
                 
        except tweepy.TweepError as e: 
            print("Error : " + str(e)) 
    def on_error(self, status):
        print (status)


if __name__ == '__main__':
    l = StdOutListener(8000)
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = tweepy.Stream(auth, l,tweet_mode='extended')
    stream.filter(languages=["en"],track=["Game","Thrones"])

closed
closed
closed
closed
closed
closed
closed
closed


SystemExit: Limit reached